In [6]:
import sklearn

#NN Surrogate model class
import injector_surrogate_quads
from injector_surrogate_quads import *
sys.path.append('../configs')
#Sim reference point to optimize around
from ref_config import ref_point

#BO
from bayes_opt import BayesianOptimization

# Load injector model

In [7]:
Model = Surrogate_NN()

Model.load_saved_model(model_path = '../models/', \
                       model_name = 'model_OTR2_NA_rms_emit_elu_2021-07-27T19_54_57-07_00')
Model.load_scaling()
Model.take_log_out = False

2022-06-01 11:37:07.498276: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Set up sampling and objectives

In [8]:
#convert to machine units
ref_point = Model.sim_to_machine(np.asarray(ref_point))

#input params: solenoid and quads to vary 
opt_var_names = ['SOL1:solenoid_field_scale','SQ01:b1_gradient','CQ01:b1_gradient']

#output params: emittance in transverse plane (x & y)
opt_out_names = ['norm_emit_x','norm_emit_y']

def evaluate(varx,vary,varz): 

    #make input array of length model_in_list (inputs model takes)
    x_in = np.empty((1,len(Model.model_in_list)))

    #fill in reference point around which to optimize
    x_in[:,:] = np.asarray(ref_point[0])

    #set solenoid, SQ, CQ to values from optimization step
    x_in[:, Model.loc_in[opt_var_names[0]]] = varx
    x_in[:, Model.loc_in[opt_var_names[1]]] = vary
    x_in[:, Model.loc_in[opt_var_names[2]]] = varz

    #output predictions
    y_out = Model.pred_machine_units(x_in) 

    return -1*objective(y_out)[0]


def objective(y_out):
    
    #output is geometric emittance in transverse plane
    out1 = y_out[:,Model.loc_out['norm_emit_x']] #grab norm_emit_x out of the model
    out2 = y_out[:,Model.loc_out['norm_emit_y']] #grab norm_emit_y out of the model
       
    return np.sqrt(out1*out2)/1e-6 # in um units

# Simple BO

In [9]:
# bounds on input params 
pbounds = {'varx': (0.44, 0.55),
           'vary': (-0.02, 0.02),
           'varz': (-0.02, 0.02)
          }

optimizer = BayesianOptimization(
    f = evaluate,
    pbounds = pbounds,
    random_state = 1,
)

optimizer.maximize(
    init_points=5,
    n_iter=40,
)

|   iter    |  target   |   varx    |   vary    |   varz    |
-------------------------------------------------------------


2022-06-01 11:37:33.755346: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-06-01 11:37:33.933460: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1996250000 Hz


|  1        | -1.15     |  0.4859   |  0.008813 | -0.02     |
|  2        | -1.105    |  0.4733   | -0.01413  | -0.01631  |
|  3        | -1.208    |  0.4605   | -0.006178 | -0.004129 |
|  4        | -2.018    |  0.4993   | -0.003232 |  0.007409 |
|  5        | -1.036    |  0.4625   |  0.01512  | -0.0189   |
|  6        | -2.511    |  0.4425   | -0.01803  | -0.01976  |
|  7        | -1.155    |  0.4594   |  0.01383  | -0.01832  |
|  8        | -0.9866   |  0.4703   |  0.02     | -5.922e-0 |
|  9        | -0.9841   |  0.465    | -0.01378  |  0.01789  |
|  10       | -2.227    |  0.4521   |  0.01222  |  0.02     |
|  11       | -1.301    |  0.4917   | -0.00306  | -0.001123 |
|  12       | -0.5854   |  0.4755   |  0.004198 | -0.005561 |
|  13       | -2.319    |  0.4938   |  0.01365  |  0.01916  |
|  14       | -0.9201   |  0.4755   |  0.01054  |  0.01428  |
|  15       | -1.163    |  0.4593   |  0.01298  | -0.01783  |
|  16       | -6.234    |  0.55     | -0.02     | -0.02     |
|  17   

## Results from simple BO

In [10]:
SOL_opt = optimizer.max['params']['varx'] # solenoid val at optimum
CQ_opt = optimizer.max['params']['vary'] # CQ val at optimum
SQ_opt = optimizer.max['params']['varz'] # SQ val at optimum

opt_emit = -1*optimizer.max['target'] # emittance value at optimum (in um)

print('optimum (pv_units) ',SOL_opt, CQ_opt, SQ_opt)
print('optimum geom emit ', opt_emit)

optimum (pv_units)  0.4780202706676465 0.00031806251471239513 -0.0007734038723606375
optimum geom emit  0.5590128898620605
